In [ ]:
#Calculate the maximum charge in kWh that each vehicle can be delivered to each vehicle to based on how long it is plugged 
#in and state the intial charge. Here it is assumed the inital charge is the same for every vehicle and that each
#vehicle can be charged up to a max of 25KWh.
for i in range(len(df)):
    time_in = int(math.ceil(df['Enter'].iloc[i]))
    time_out = int(math.floor(df['Leave'].iloc[i]))
#Assume every vehicle has an intial charge of 12.5

    Initial_charge[i] = 12.5
    if time_out > time_in:
#Calculate the duration of the stay
        length_stay[i] = time_out - time_in
#Calculate the maximum amount of charge that can be delivered to the vehicle
        charge_if_max_power_for_length_stay[i] = (7/60) * length_stay[i]
        if charge_if_max_power_for_length_stay[i] > 25 - Initial_charge[i]:
            max_potential_charge[i] = round_down(25 - Initial_charge[i],2)
        elif charge_if_max_power_for_length_stay[i] < 25 - Initial_charge[i]:
            max_potential_charge[i] = round_down(charge_if_max_power_for_length_stay[i],2)

    elif time_out < time_in:
        length_stay[i] = (1440 - time_in) + time_out 
        charge_if_max_power_for_length_stay[i] = (7/60) * length_stay[i]
        if charge_if_max_power_for_length_stay[i] > 25 - Initial_charge[i]:
            max_potential_charge[i] = round_down(25 - Initial_charge[i],2)
        elif charge_if_max_power_for_length_stay[i] < 25 - Initial_charge[i]:
            max_potential_charge[i] = round_down(charge_if_max_power_for_length_stay[i],2)

df['max_deliverable_charge'] = max_potential_charge
df['Initial_charge'] = Initial_charge
energy_desired = np.zeros(len(df))

#Calculate the total energy that needs to be delivered to each vehicle. Here we assume each user wamts to charge their 
#vehicle as much as possible
for e in range(len(df)):   
    if 25 - df['Initial_charge'].iloc[e] > df['max_deliverable_charge'].iloc[e]:
        energy_desired[e] = round_down(df['max_deliverable_charge'].iloc[e] + df['Initial_charge'].iloc[e],2)
    elif 25 - df['Initial_charge'].iloc[e] <= df['max_deliverable_charge'].iloc[e]:
        energy_desired[e] = 25
df['energy_desired']= energy_desired
df['charge_needed'] = df['energy_desired'] - df['Initial_charge']

#initialise key matrices and vectors
power = np.zeros((1440,len(df)))
energy_delivered = np.zeros(len(df))
minutes_to_charge = np.zeros((1440,len(df)))
initial_no_min_charging = np.zeros(len(df))
power_limit = 2000
total_power_per_minute = np.zeros(1440)

for C in range(len(df)):
#Get the times the vehicle plugs-in and plugs-out in minutes for indexing
    time_in = int(math.ceil(df['Enter'].iloc[C]))
    time_out = int(math.floor(df['Leave'].iloc[C]))
#Calculate the minimum number of required charging minutes
    initial_no_min_charging[C] = math.ceil(round_down(df['charge_needed'].iloc[C],2)/(7/60)-1)
#If the user chooses to prioritise time then the minutes that are required to charge the vehicle are worked out. These are the
#just the minutes between the plug-in and plug-out times in order.
     
    if df['Priority'].iloc[C] == 'Time':
        if time_out > time_in:
            for a in range(time_out-time_in+1):
                minutes_to_charge[a,C] = time_in+a
        elif time_in > time_out:
            for a in range((1440 - time_in) + time_out):
                if a + time_in < 1440:
                    minutes_to_charge[a,C] = a + time_in
                elif a + time_in >= 1440:
                    minutes_to_charge[a,C] = a + time_in - 1440 + 1
#If the user chooses to prioritise coat then the minutes that are required to charge the vehicle are worked out. These are the
#just the cheapest minutes between the plug-in and plug-out times in order.            
    elif df['Priority'].iloc[C] == 'Cost':
        if time_out > time_in:
            sorted_arguments = np.argsort(cost[time_in:time_out+1])
            for b in range(len(sorted_arguments)):
                minutes_to_charge[b,C] = sorted_arguments[b] + time_in 
        elif time_in > time_out:
            
            array1 = cost[time_in:]
            array2 = cost[:time_out+1]
            array3 = np.append(array1,array2)
            sorted_arguments = np.argsort(array3)
            for c in range(len(sorted_arguments)):
                if sorted_arguments[c] + time_in < 1440:
                    minutes_to_charge[c,C] = sorted_arguments[c] + time_in
                elif sorted_arguments[c] + time_in >= 1440:
                    minutes_to_charge[c,C] = sorted_arguments[c] + time_in - 1440 

                
    #print(initial_no_min_charging[C])
    length = int(initial_no_min_charging[C])

#Charging at maximum power for the required minimum number of minutes
    power_max = 7            
    for d in range(length):
        index = int(minutes_to_charge[d,C])
        power[index,C] = power_max
#CHarging up to final desired charge    
    final_index = int(minutes_to_charge[length+1,C])
    power[final_index,C] = (df['charge_needed'].iloc[C] - np.sum(power[:,C])/60)*60  
    energy_delivered[C] = np.sum(power[:,C])/60   
    
for e in range(1440):
    total_power_per_minute[e] = np.sum(power[e,:])
total_power = np.zeros(1440)
#total_power vector scaled to the Graden village
for f in range(1440):
    total_power[f] = np.sum(power[f,:])
        

no_min_charging = np.zeros(len(df))
for E in range(len(df)):
    no_min_charging[E] = initial_no_min_charging[E]
#POWER LIMITING SECTION OF THE ALGORITHM   
condition = all(X <= power_limit for X in total_power)
Z = 0
while condition == False:  

#For all minutes where the total power is over the limit the limited power is split between the active chargers
    for f in range(1440):
        if total_power_per_minute[f] > power_limit:
            number_chargers_active = np.count_nonzero(power[f,:])
            new_power_max = power_limit/number_chargers_active
            for g in range(len(df)):
                if power[f,g] > 0:
                    power[f,g] = new_power_max
                    if np.sum(power[:,g])/60 > df['charge_needed'].iloc[g]:
                        power[f,g] = 0
                        power[f,g] = (df['charge_needed'].iloc[g]-np.sum(power[:,g])/60)*60
                                       
#Using unused minutes to charge the vehicles to their desired level    
    for h in range(len(df)):
        for H in range(1440):
            if np.sum(power[:,h])/60 < df['charge_needed'].iloc[h]:
                index = int(minutes_to_charge[H,h])
                if power[index,h] == 0:
                    power[index,h] = power_max
                    if np.sum(power[:,h])/60 > df['charge_needed'].iloc[h]:
                            power[index,h] = 0
                            power[index,h] = (df['charge_needed'].iloc[h]-np.sum(power[:,h])/60)*60

                
            
        if (np.sum(power[:,h]) < round_down(df['charge_needed'].iloc[h],2)) & (round_down(df['charge_needed'].iloc[h],2) <12.5):
            df['charge_needed'].iloc[h] == np.sum(power[:,h])
            print(df['charge_needed'].iloc[h])            
                
    for i in range(1440):
        total_power_per_minute[i] = int(np.sum(power[i,:]))
    total_power = np.zeros(1440)
    for f in range(1440):
        total_power[f] = np.sum(power[f,:])
    Z = Z+1
    print(Z)
    print(max(total_power))
    condition = all(Y <= 2000 for Y in total_power)